# Neural Texture Transfer

In [0]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (16,20)
mpl.rcParams['axes.grid'] = False
import numpy as np
from PIL import Image
import scipy.misc

import tensorflow as tf
import tensorflow.contrib.eager as tfe
from tensorflow.python.keras.preprocessing import image as kp_image
from tensorflow.python.keras import models
#from TextUtils import *
from PIL import Image
from tensorflow.python.keras.preprocessing import image as k_process
import IPython.display

In [2]:
tf.enable_eager_execution()
print("Eager execution enabled: {}".format(tf.executing_eagerly()))

Eager execution enabled: True


In [0]:
content_path = 'cathd.jpg'; texture_path = 'carbon crack.jpg'

In [0]:
def slicer(path_to_img, mode = None, size = None, split = (3,3)):
    img = Image.open(path_to_img)
    if mode is "texture":
        img = img.resize(size).convert('L').convert('RGB')
    imgwidth, imgheight = img.size
    n_split_w, n_split_h = split
    height = imgheight//n_split_h; width = imgwidth//n_split_w
    #print('slice', height, width)
    imgs = []
    for i in range(0,n_split_h*height,height):
        for j in range(0,n_split_w*width,width):
            box = (j, i, j+width, i+height)
            imgs.append(np.array(img.crop(box)))
    return np.array(imgs).astype('uint8')
  
 
def recomp(img):
  #tf, np = imp()
  up = np.concatenate(img[:3], axis = 1)
  mid = np.concatenate(img[3:6], axis = 1)
  down = np.concatenate(img[6:], axis = 1)
  #print(up.shape, mid.shape, down.shape)
  tout = np.concatenate([up ,mid, down], axis = 0)
  tout = Image.fromarray(tout)
  return tout

In [0]:
def turn_bw(img):
    img = tf.image.rgb_to_grayscale(img)
    img_bw = tf.image.grayscale_to_rgb(img)
    return img_bw

In [0]:
def get_model(texture_layers, content_layers):
  #from tensorflow.python.keras import models
  #tf, np = imp()
  # Load our model. We load pretrained VGG, trained on imagenet data
  vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False
  # Get output layers corresponding to style and content layers
  texture_outputs = [vgg.get_layer(name).output for name in texture_layers]
  content_outputs = [vgg.get_layer(name).output for name in content_layers]
  model_outputs = texture_outputs + content_outputs
  # Build model
  return models.Model(vgg.input, model_outputs)

In [0]:
def deprocess_img(processed_img):
  #tf, np = imp()
  x = processed_img.copy()
  if len(x.shape) == 0:
    x = np.squeeze(x, 0)
  assert len(x.shape) == 4, ("Input to deprocess image must be an image of "
                             "dimension [1, height, width, channel] or [height, width, channel]")
  if len(x.shape) != 4:
    raise ValueError("Invalid input to deprocessing image")

  # perform the inverse of the preprocessiing step
  x[:, :, :, 0] += 103.939
  x[:, :, :, 1] += 116.779
  x[:, :, :, 2] += 123.68
  x = x[:, :, :, ::-1]

  x = np.clip(x, 0, 255).astype('uint8')
  return x

In [0]:
def load_img(img):
  #from PIL import Image
  #from tensorflow.python.keras.preprocessing import image as k_process
  #tf, np = imp()
  max_dim = 512
  #img = Image.open(path_to_img)
  long = max(img.shape)
  b, h, w, c = img.shape
  scale = max_dim/long
  hr , wr = round(h*scale), round(w*scale)
  #print('hr', hr, wr)
  ar = k_process.img_to_array; pics = Image.fromarray
  img = np.array([ar(pics(d).resize((wr, hr), Image.ANTIALIAS)) for d in img])

  return img


def load_and_process_img(img):
  #tf, np = imp()
  img = load_img(img)
  img = tf.keras.applications.vgg19.preprocess_input(img)
  return img

In [0]:
def gram_matrix(input_tensor):
    batch,  height, width, channels = [int(d) for d in input_tensor.shape]
    tensor = tf.reshape(input_tensor, [batch, height*width, channels])
    # Get the product height*width
    n = tf.shape(tensor)[1]
    gram = tf.matmul(tensor, tensor, transpose_a=True)
    #gram = tf.nn.dropout(gram, 0.5, seed = 1)
    #gram = tf.nn.max_pool(tf.reshape(gram,[1,channels,channels,1]),ksize = [1,2, 2,1], strides=[1, 1, 1, 1],padding= 'VALID')
    return gram / tf.cast(n, tf.float32)

In [0]:
  def get_loss(tensor1, tensor2):
    reduce_space = list(range(1,len(tensor1.shape)))
    #print(ls)
    return tf.reduce_mean(tf.square(tensor1 - tensor2), reduce_space)

In [0]:
def get_deep_outputs(model, inputs, num_texture_layers):
    
    t = int(inputs.shape[0])//2
    outputs = model(inputs)
    color_outputs = [layer[:t] for layer in outputs[num_texture_layers:]]
    bw_outputs = [gram_matrix(layer[t:2*t]) for layer in outputs[:num_texture_layers]]
    #img_outputs = [layer[2*t:3*t] for layer in outputs[num_texture_layers:]]
    #img_bw_outputs = [gram_matrix(layer[3*t:]) for layer in outputs[:num_texture_layers]]
    
    return color_outputs, bw_outputs

In [0]:
def compute_loss(model, loss_weights, img_sync, const_outputs, content_image):
    
    content_output, texture_output  = const_outputs
    num_texture_layers = len(texture_output)
    num_content_layers = len(content_output)
    
    img_sync_bw = turn_bw(img_sync) # copy!!!
    var_inputs = tf.concat([img_sync, img_sync_bw], axis = 0)
    img_sync_output, img_sync_bw_output = get_deep_outputs(model, var_inputs, num_texture_layers)
    
    texture_loss = 0
    content_loss = 0
    
    # Accumulate texture losses from all layers
    w_text = 1.0 / float(num_texture_layers)
    for texture_feat, var_bw_feat in zip(texture_output, img_sync_bw_output):
        texture_loss += w_text * get_loss(texture_feat, var_bw_feat)
    
    # Accumulate content losses from all layers
    w_cont = 1.0 / float(num_content_layers)
    for content_feat, var_feat in zip(content_output, img_sync_output):
        content_loss += w_cont * get_loss(content_feat, var_feat)
    
    direct_weight, texture_weight, content_weight = loss_weights
    texture_loss *= texture_weight
    content_loss *= content_weight
    
    direct_loss = direct_weight*get_loss(img_sync, content_image)
   
    total_loss = texture_loss + content_loss + direct_loss
    
    return total_loss

In [0]:
def get_grads(args):
  with tf.GradientTape() as tape: 
    loss = compute_loss(**args)
  return tape.gradient(loss, args['img_sync']), loss

In [0]:
content_path = 'cathd.jpg'; texture_path = 'carbon crack.jpg'

In [0]:
content_layers = ['block1_conv1', 'block5_conv2'] 
content_layers = ['block5_conv2']
# Style layer we are interested in
texture_layers = ['block1_conv1',
                  'block2_conv1'#,'block3_conv1'#, 'block4_conv1'#, 'block5_conv1'
               ]


In [0]:
def neural_texture_train(content_path, texture_path, split = (2,2),loss_weights = (1e3,1e3, 1e-3), num_iterations=1000,
                        layers = ()):
    
    texture_layers, content_layers = layers
    num_content_layers = len(content_layers)
    num_texture_layers = len(texture_layers)
    
    model = get_model(texture_layers, content_layers) 
    for layer in model.layers:
        layer.trainable = False
        
    content_image = slicer(content_path, split=split)
    content_image = load_and_process_img(content_image)
    
    size = Image.open(content_path).size
    texture_image = slicer(texture_path, size = size, mode='texture', split=split)
    texture_image = load_and_process_img(texture_image)
    
    img_sync = tfe.Variable(content_image, dtype=tf.float32)
    
    
    const_inputs = tf.concat([content_image, texture_image], axis = 0)
    const_outputs = get_deep_outputs(model, const_inputs, num_texture_layers)
    
    opt = tf.train.AdamOptimizer(learning_rate=5, beta1=0.99, epsilon=1e-1)
    
    args = {'model': model,'loss_weights': loss_weights,'img_sync': img_sync, 
            'const_outputs': const_outputs, 'content_image': content_image, }
    
    norm_means = np.array([103.939, 116.779, 123.68])# norms(init_image.shape)
    min_vals = -norm_means
    max_vals = 255 - norm_means
    display_interval = 10
    for i in range(num_iterations):
        print('Iteration ', i)
        grads, loss = get_grads(args)
        opt.apply_gradients([(grads, img_sync)])
        clipped = tf.clip_by_value(img_sync, min_vals, max_vals)
        img_sync.assign(clipped)
        
        if i % display_interval == 0:
            # Use the .numpy() method to get the concrete numpy array
            plot_img = img_sync.numpy()
            plot_img = deprocess_img(plot_img)
            #scipy.misc.imsave('outfile.jpg', recomp(plot_img))
            IPython.display.clear_output(wait=False)
            IPython.display.display_png(Image.fromarray(plot_img[1]))
            print('Iteration: {}'.format(i))        
            print('Total loss: {:.4e}, ' )
    
    pics = deprocess_img(img_sync.numpy())
    return recomp(pics)
    
    

In [0]:
neural_texture_train(content_path, texture_path, loss_weights = (5e3,1e3, 1e-3), num_iterations=1000,
                        split = (3,3), layers = (content_layers, texture_layers))